In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

## Prompting Techniques

### Zero-Shot Prompting

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Translate to French: {sentence}")
response = model.invoke(prompt.format(sentence="How are you?"))
print(response.content)

There are a few ways to translate "How are you?" into French, depending on the level of formality:

*   **Formal:** **Comment allez-vous ?** (This is the most formal and polite option, using the formal "vous" form.)

*   **Informal:** **Comment vas-tu ?** (This is used with friends, family, or people you know well, using the informal "tu" form.)

*   **Very informal:** **Ça va ?** (This is a very common and casual way to ask, similar to "You okay?" or "What's up?")

*   **Common, slightly less informal than "Ça va ?":** **Comment ça va ?** (This is also very common and can be used in many situations, but it's slightly less informal than just "Ça va ?")

So, the best translation depends on the context. If you're unsure, **"Comment allez-vous ?"** is always a safe bet.


### Few-Shot Prompting

In [2]:
prompt = PromptTemplate.from_template("""
Translate English to French:
- Hello → Bonjour
- Good night → Bonne nuit
- {text} →""")

response = model.invoke(prompt.format(text="Thank you"))
print(response.content)

Merci


### Chain-of-Thought (CoT) Prompting

#### Zero-shot-CoT:

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

prompt = PromptTemplate.from_template("""
Solve the following problem.

Question: Sarah has 12 apples. She gives away 4, then buys 7 more.
How many apples does she have now?

Answer: Let's think step by step.
""")

response = model.invoke(prompt.format())
print(response.content)

Okay, let's break it down:

*   **Start:** Sarah begins with 12 apples.
*   **Gives away:** She gives away 4 apples, so we subtract: 12 - 4 = 8 apples.
*   **Buys more:** She buys 7 more apples, so we add: 8 + 7 = 15 apples.

**Answer:** Sarah has 15 apples now.


#### Few-shot-CoT

In [6]:
prompt = PromptTemplate.from_template("""
Solve each of the following problems step-by-step.

Q: Tom had 10 candies. He ate 3 and bought 2 more. How many candies does he have now?  
A: Step 1: Start with 10.  
Step 2: Subtract 3 → 10 - 3 = 7  
Step 3: Add 2 → 7 + 2 = 9  
Answer: 9

Q: Sarah has 12 apples. She gives 4 away, then buys 7 more. How many apples does she have now?  
A:""")

response = model.invoke(prompt.format())
print(response.content)

Step 1: Start with 12.
Step 2: Subtract 4 → 12 - 4 = 8
Step 3: Add 7 → 8 + 7 = 15
Answer: 15


### Tree-of-Thought (ToT) Prompting

In [7]:

# Define the problem
question = "Find a number between 1 and 100 that is divisible by both 4 and 6."

# Generate multiple reasoning paths
thought_prompts = [
    f"Let's solve it this way: Start listing numbers divisible by 4.",
    f"Try a different path: List numbers divisible by 6 first.",
    f"Try finding the Least Common Multiple (LCM) of 4 and 6."
]

# Collect candidate answers
candidates = []
for thought in thought_prompts:
    response = model.invoke(f"{question}\n{thought}")
    candidates.append(response.content)

# Evaluate answers (simple heuristic: check if response includes correct number)
final_answers = [ans for ans in candidates if "12" in ans or "divisible by 12" in ans]

# Choose the best one
best = final_answers[0] if final_answers else "No clear answer found"
print(best)


Okay, let's do that.

Multiples of 4: 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100

Now, let's check which of these are also divisible by 6:

*   12 / 6 = 2
*   24 / 6 = 4
*   36 / 6 = 6
*   48 / 6 = 8
*   60 / 6 = 10
*   72 / 6 = 12
*   84 / 6 = 14
*   96 / 6 = 16

So, the numbers between 1 and 100 that are divisible by both 4 and 6 are: 12, 24, 36, 48, 60, 72, 84, and 96.

The smallest of these is 12. So, a number between 1 and 100 that is divisible by both 4 and 6 is 12.

Answer: 12


In [11]:
from langchain.chains import LLMChain, SequentialChain

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Step 1: Generate Possible Solutions
prompt1 = PromptTemplate.from_template("""
You're helping someone reduce their daily screen time.

Step 1:
Come up with three different strategies they could try. Be practical and consider different types of solutions (behavioral, environmental, or technical).

A:""")
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="ideas")

# Step 2: Evaluate Each Strategy
prompt2 = PromptTemplate.from_template("""
Step 2:
Evaluate the pros and cons of each of the following strategies to reduce screen time:

{ideas}

Include ease of implementation, effectiveness, and sustainability in your evaluation.

A:""")
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="evaluations")

# Step 3: Choose the Best Strategy
prompt3 = PromptTemplate.from_template("""
Step 3:
Based on the evaluations, pick the most promising strategy for reducing screen time. Justify your choice in a few sentences.

{evaluations}

A:""")
chain3 = LLMChain(llm=llm, prompt=prompt3, output_key="final_choice")


In [12]:
# Combine into a Sequential Chain
tot_chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=[],
    output_variables=["final_choice"],
    verbose=True
)

# Run the chain
response = tot_chain({})
print(response["final_choice"])



> Entering new SequentialChain chain...

> Finished chain.
**Based on the evaluations, I recommend starting with the "Screen-Free Zone & Designated Device" approach.** This strategy offers a good balance of ease of implementation, effectiveness, and sustainability. It's relatively simple to set up, doesn't rely heavily on willpower, and can create a more mindful environment that naturally reduces screen time exposure. While it might not be a complete solution, it provides a solid foundation for further adjustments and the potential integration of other strategies if needed.


## Prompt Templates

In [3]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import PromptTemplate

llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    region_name="us-west-2",
    temperature=0,
    # max_tokens=1024 
) 

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv

load_dotenv()  # loads the .env file

### PromptTemplates

In [4]:
prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

# fill the values of the placeholders
prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

text='Tell me a joke about cats'


### ChatPromptTemplate

In [5]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate([
    ('system', 'You are a helpful {domain} expert'),
    ('human', 'Explain in simple terms, what is {topic}')
])

prompt = chat_template.invoke({'domain':'cricket','topic':'Googly'})

print(prompt)

messages=[SystemMessage(content='You are a helpful cricket expert', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain in simple terms, what is Googly', additional_kwargs={}, response_metadata={})]


### Message Placeholders:

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_history = []
# load chat history
with open('chat_history.txt') as f:
    chat_history.extend(f.readlines())
    
# chat template
chat_template = ChatPromptTemplate([
    ('system','You are a helpful customer support agent'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human','{query}')
])

# create prompt
prompt = chat_template.invoke({'chat_history':chat_history, 'query':'When can I expect my order?'})

result = llm.invoke(prompt)
print(result.content)

Based on the information provided, your order #435 has been prepared for shipping and will be delivered within 3-5 business days. This means you should receive your order between Monday and Wednesday next week, assuming today is a business day. Would you like me to provide you with the tracking information once it becomes available?


### Few-Shot Prompts:

In [9]:
from langchain.prompts import FewShotPromptTemplate

examples = [
    {"word": "hello", "synonym": "hi"},
    {"word": "happy", "synonym": "joyful"}
]

example_template = PromptTemplate.from_template("Word: {word}\nSynonym: {synonym}\n")
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="Give a synonym for each word:",
    suffix="Word: {input}\nSynonym:",
    input_variables=["input"]
)

formatted_prompt = prompt.format(input="sad")
output = llm.invoke(formatted_prompt)

print("Prompt:\n", formatted_prompt)
print("-"*50)
print("Output:", output.content)

Prompt:
 Give a synonym for each word:

Word: hello
Synonym: hi


Word: happy
Synonym: joyful


Word: sad
Synonym:
--------------------------------------------------
Output: Word: sad
Synonym: unhappy


### Message Template:

In [14]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatPromptTemplate
)

# System message for context
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an expert {field} consultant with {years} years of experience."
)

# Human message for the query
human_prompt = HumanMessagePromptTemplate.from_template(
    "I need help with {problem}. My current situation is: {context}"
)

# AI message for providing examples of responses
ai_prompt = AIMessagePromptTemplate.from_template(
    "I understand you need help with {problem}. Let me analyze your situation."
)

# Combine all messages
full_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    human_prompt,
    ai_prompt
])

In [16]:
# Provide input values for all placeholders
formatted_prompt = full_prompt.invoke({
    "field": "marketing",
    "years": "10",
    "problem": "low conversion rates on my website",
    "context": "We're getting a lot of traffic, but very few visitors are signing up for our product."
})

# Print each message from the formatted prompt
for role, content in formatted_prompt:
    print(f"{role.upper()}: {content}")

MESSAGES: [SystemMessage(content='You are an expert marketing consultant with 10 years of experience.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I need help with low conversion rates on my website. My current situation is: We're getting a lot of traffic, but very few visitors are signing up for our product.", additional_kwargs={}, response_metadata={}), AIMessage(content='I understand you need help with low conversion rates on my website. Let me analyze your situation.', additional_kwargs={}, response_metadata={})]


### Partial Prompts:

In [17]:
prompt = PromptTemplate(
    template="You are a {role} helping with {task} in {domain}",
    input_variables=["role", "task", "domain"]
)

# Partially fill the prompt
partial_prompt = prompt.partial(role="senior developer")

# Later, complete the remaining variables
final_prompt = partial_prompt.format(
    task="code review",
    domain="web development"
)

# Output the final prompt
print(final_prompt)

You are a senior developer helping with code review in web development
